In [3]:
import ctypes
import ctypes.util
import importlib
import math
import os
import pathlib
import time

import cffi
import dolfinx
import numba
import numba.core.typing.cffi_utils as cffi_support
import numpy as np
import petsc4py.lib
#import pytest
import ufl
#from dolfinx.jit import dolfinx_pc
import dolfinx.jit
#from dolfinx.jit import dolfinx_pc
from mpi4py import MPI
from petsc4py import PETSc
from petsc4py import get_config as PETSc_get_config
from ufl import dx, inner


ImportError: cannot import name 'dolfinx_pc' from 'dolfinx.jit' (/Users/markloveland/miniforge3/envs/fenicsx-env/lib/python3.10/site-packages/dolfinx/jit.py)

In [2]:
print(dir(dolfinx.jit))

['DOLFINX_DEFAULT_JIT_PARAMETERS', 'MPI', 'Optional', 'Path', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_load_parameters', 'ffcx', 'ffcx_jit', 'functools', 'get_parameters', 'json', 'mpi_jit_decorator', 'os', 'ufl']


In [9]:

mesh1 = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, 10, 10,dolfinx.mesh.CellType.triangle )
mesh2 = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, 10, 10,dolfinx.mesh.CellType.triangle )
V1 = dolfinx.fem.FunctionSpace(mesh1, ("CG", 1))
u1, v1 = ufl.TrialFunction(V1), ufl.TestFunction(V1)
V2 = dolfinx.fem.FunctionSpace(mesh2, ("CG", 1))
u2, v2 = ufl.TrialFunction(V2), ufl.TestFunction(V2)

#in an ideal world can just do this
a = u1*v1*u2*v2*ufl.dx(domain=mesh1)*ufl.dx(domain=mesh2)

TypeError: unsupported operand type(s) for *: 'Form' and 'Measure'

In [3]:
# (Maybe useful in future)
# Prepare a marking structures
# indices cover all cells 
# values are [1, 2, 3, 3, ...]
cell_map = mesh.topology.index_map(mesh.topology.dim)
num_cells = cell_map.size_local + cell_map.num_ghosts
#indices = numpy.arange(0, num_cells)

NameError: name 'mesh' is not defined

In [4]:
@numba.njit(fastmath=True)
def assemble_matrix_cffi(A, mesh, dofmap, num_cells, set_vals, mode):
    """Assemble P1 mass matrix over a mesh into the PETSc matrix A"""

    # Mesh data
    v, x = mesh

    # Quadrature points and weights
    q = np.array([[0.5, 0.0], [0.5, 0.5], [0.0, 0.5]], dtype=np.double)
    weights = np.full(3, 1.0 / 3.0, dtype=np.double)

    # Loop over cells
    N = np.empty(3, dtype=np.double)
    A_local = np.empty((3, 3), dtype=PETSc.ScalarType)
    for cell in range(num_cells):
        cell_area = area(x[v[cell, 0]], x[v[cell, 1]], x[v[cell, 2]])

        # Loop over quadrature points
        A_local[:] = 0.0
        for j in range(q.shape[0]):
            N[0], N[1], N[2] = 1.0 - q[j, 0] - q[j, 1], q[j, 0], q[j, 1]
            for row in range(3):
                for col in range(3):
                    A_local[row, col] += weights[j] * cell_area * N[row] * N[col]

        # Add to global tensor
        pos = dofmap[cell, :]
        set_vals(A, 3, ffi.from_buffer(pos), 3, ffi.from_buffer(pos), ffi.from_buffer(A_local), mode)
    sink(A_local, dofmap)